# Exploratory Data Analysis - Kickstarter Projects

This notebook explores the Kickstarter project datasets to understand:
- Data structure and quality
- Project success rates
- Funding patterns
- Category and country distributions
- Temporal trends


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


## 1. Load Data


In [ ]:
# Define data paths
data_dir = Path('../../data/raw')
file_201612 = data_dir / 'ks-projects-201612.csv'
file_201801 = data_dir / 'ks-projects-201801.csv'

# Load datasets with encoding handling
print("Loading datasets...")

# Try different encodings if UTF-8 fails
encodings = ['utf-8', 'latin-1', 'ISO-8859-1', 'cp1252']

def load_csv_with_encoding(file_path):
    """Try to load CSV with different encodings"""
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding, low_memory=False)
            print(f"Successfully loaded {file_path.name} with {encoding} encoding")
            return df
        except UnicodeDecodeError:
            continue
    # If all encodings fail, try with encoding_errors parameter (pandas 1.3+)
    try:
        print(f"Warning: Using utf-8 with encoding_errors='replace' for {file_path.name}")
        return pd.read_csv(file_path, encoding='utf-8', encoding_errors='replace', low_memory=False)
    except TypeError:
        # Fallback for older pandas versions - use latin-1 which can read any byte
        print(f"Warning: Using latin-1 encoding for {file_path.name} (fallback)")
        return pd.read_csv(file_path, encoding='latin-1', low_memory=False)

df_201612 = load_csv_with_encoding(file_201612)
df_201801 = load_csv_with_encoding(file_201801)

print(f"\n201612 dataset shape: {df_201612.shape}")
print(f"201801 dataset shape: {df_201801.shape}")


## 2. Data Overview


In [ ]:
# Display column names and basic info
print("=" * 60)
print("201612 Dataset Columns:")
print("=" * 60)
print(df_201612.columns.tolist())
print(f"\nNumber of columns: {len(df_201612.columns)}")

print("\n" + "=" * 60)
print("201801 Dataset Columns:")
print("=" * 60)
print(df_201801.columns.tolist())
print(f"\nNumber of columns: {len(df_201801.columns)}")


In [ ]:
# Display first few rows
print("=" * 60)
print("201612 Dataset - First 5 rows:")
print("=" * 60)
df_201612.head()


In [ ]:
print("=" * 60)
print("201801 Dataset - First 5 rows:")
print("=" * 60)
df_201801.head()


## 3. Data Cleaning and Preparation


In [ ]:
# Clean column names (remove spaces)
df_201612.columns = df_201612.columns.str.strip()
df_201801.columns = df_201801.columns.str.strip()

# Drop unnamed columns from 201612 dataset
unnamed_cols = [col for col in df_201612.columns if 'Unnamed' in col]
if unnamed_cols:
    print(f"Dropping {len(unnamed_cols)} unnamed columns from 201612 dataset")
    df_201612 = df_201612.drop(columns=unnamed_cols)

# Check for missing values
print("=" * 60)
print("Missing Values - 201612 Dataset:")
print("=" * 60)
missing_201612 = df_201612.isnull().sum()
print(missing_201612[missing_201612 > 0])

print("\n" + "=" * 60)
print("Missing Values - 201801 Dataset:")
print("=" * 60)
missing_201801 = df_201801.isnull().sum()
print(missing_201801[missing_201801 > 0])


In [ ]:
# Convert date columns first to check date ranges
df_201612['launched'] = pd.to_datetime(df_201612['launched'], errors='coerce')
df_201612['deadline'] = pd.to_datetime(df_201612['deadline'], errors='coerce')
df_201801['launched'] = pd.to_datetime(df_201801['launched'], errors='coerce')
df_201801['deadline'] = pd.to_datetime(df_201801['deadline'], errors='coerce')

# Combine datasets - use the more complete 201801 dataset as primary
# Check if we should merge or use separately
print("=" * 60)
print("Dataset Comparison:")
print("=" * 60)
valid_dates_201612 = df_201612['launched'].dropna()
valid_dates_201801 = df_201801['launched'].dropna()

if len(valid_dates_201612) > 0:
    print(f"201612 date range: {valid_dates_201612.min()} to {valid_dates_201612.max()}")
    print(f"201612 valid dates: {len(valid_dates_201612)}/{len(df_201612)} ({len(valid_dates_201612)/len(df_201612)*100:.1f}%)")
else:
    print("201612: No valid dates found")

if len(valid_dates_201801) > 0:
    print(f"201801 date range: {valid_dates_201801.min()} to {valid_dates_201801.max()}")
    print(f"201801 valid dates: {len(valid_dates_201801)}/{len(df_201801)} ({len(valid_dates_201801)/len(df_201801)*100:.1f}%)")
else:
    print("201801: No valid dates found")

# Use the more recent dataset (201801) which has more complete columns
df = df_201801.copy()
print(f"\nUsing 201801 dataset with {len(df)} rows")


In [ ]:
# Check for projects with inconsistent data
print("=" * 60)
print("DATA QUALITY SUMMARY:")
print("=" * 60)

# Create flags for data quality issues
df['has_zero_backers'] = df['backers'] == 0
df['has_zero_pledged'] = df['usd_pledged_real'] == 0
df['has_zero_goal'] = df['usd_goal_real'] == 0
# Only check invalid duration if the column exists (created in Cell 16)
if 'campaign_duration_days' in df.columns:
    df['has_invalid_duration'] = df['campaign_duration_days'] <= 0
else:
    df['has_invalid_duration'] = False  # Set to False if column doesn't exist yet
df['has_data_inconsistency'] = ((df['usd_pledged_real'] > 0) & (df['backers'] == 0)) | \
                                 ((df['state'] == 'successful') & (df['usd_pledged_real'] == 0))

total_issues = df['has_data_inconsistency'].sum()
print(f"\nProjects with data inconsistencies: {total_issues:,} ({total_issues/len(df)*100:.2f}%)")

# Show breakdown
if total_issues > 0:
    print("\nBreakdown of inconsistencies:")
    inconsistent = df[df['has_data_inconsistency']]
    print(f"  - Pledged > 0 but backers = 0: {((inconsistent['usd_pledged_real'] > 0) & (inconsistent['backers'] == 0)).sum():,}")
    print(f"  - Successful but pledged = 0: {((inconsistent['state'] == 'successful') & (inconsistent['usd_pledged_real'] == 0)).sum():,}")


In [ ]:
# Dates are already converted, but filter out invalid dates
# Filter out dates before 2009 (Kickstarter started in 2009) and after 2018
# Also filter out the default Unix epoch date (1970-01-01)
min_valid_date = pd.Timestamp('2009-01-01')
max_valid_date = pd.Timestamp('2018-12-31')

# Count invalid dates before filtering
invalid_dates = ((df['launched'] < min_valid_date) | (df['launched'] > max_valid_date) | df['launched'].isna()).sum()
print(f"Invalid dates (before 2009, after 2018, or NaN): {invalid_dates} ({invalid_dates/len(df)*100:.2f}%)")

# Create derived features
df['campaign_duration_days'] = (df['deadline'] - df['launched']).dt.days
df['year_launched'] = df['launched'].dt.year
df['month_launched'] = df['launched'].dt.month
df['day_of_week'] = df['launched'].dt.day_name()

# Calculate funding ratio (pledged / goal)
df['funding_ratio'] = df['usd_pledged_real'] / df['usd_goal_real']
df['funding_ratio'] = df['funding_ratio'].replace([np.inf, -np.inf], np.nan)

# Filter to valid date range for analysis
df_valid_dates = df[(df['launched'] >= min_valid_date) & (df['launched'] <= max_valid_date)].copy()

print(f"\nDate conversions and derived features created.")
print(f"Original dataset: {len(df)} rows")
print(f"Valid date range: {df_valid_dates['launched'].min()} to {df_valid_dates['launched'].max()}")
print(f"Rows with valid dates: {len(df_valid_dates)} ({len(df_valid_dates)/len(df)*100:.1f}%)")

# Use df_valid_dates for the rest of the analysis
df = df_valid_dates.copy()


## 4. Basic Statistics


In [ ]:
# Basic info about the dataset
print("=" * 60)
print("Dataset Info:")
print("=" * 60)
df.info()


In [ ]:
# Statistical summary for numerical columns
print("=" * 60)
print("Numerical Summary Statistics:")
print("=" * 60)
numerical_cols = ['goal', 'pledged', 'backers', 'usd_goal_real', 'usd_pledged_real', 
                  'campaign_duration_days', 'funding_ratio']
df[numerical_cols].describe()


In [ ]:
# Project states distribution
print("=" * 60)
print("Project States Distribution:")
print("=" * 60)
state_counts = df['state'].value_counts()
print(state_counts)
print(f"\nSuccess rate: {(df['state'] == 'successful').sum() / len(df) * 100:.2f}%")


In [ ]:
# Numerical values that are zero
numerical_cols = ['goal', 'pledged', 'backers', 'usd_goal_real', 'usd_pledged_real', 
                      'campaign_duration_days', 'funding_ratio']

zero_counts = df[numerical_cols].eq(0).sum()
print("=" * 60)
print("Number of Zero Values in Each Numerical Column:")
print("=" * 60)
print(zero_counts)
print("\nColumns with the most zeros:")
print(zero_counts.sort_values(ascending=False))


## 5. Visualizations


In [ ]:
# 1. Project States Distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
state_counts = df['state'].value_counts()
axes[0].bar(state_counts.index, state_counts.values, color='steelblue')
axes[0].set_title('Project States Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('State')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# Pie chart
axes[1].pie(state_counts.values, labels=state_counts.index, autopct='%1.1f%%', startangle=90)
axes[1].set_title('Project States Proportion', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
# 2. Main Categories Distribution
plt.figure(figsize=(14, 6))
category_counts = df['main_category'].value_counts().head(15)
sns.barplot(x=category_counts.values, y=category_counts.index, palette='viridis')
plt.title('Top 15 Main Categories by Project Count', fontsize=14, fontweight='bold')
plt.xlabel('Number of Projects')
plt.ylabel('Main Category')
plt.tight_layout()
plt.show()


In [ ]:
# 3. Success Rate by Main Category
category_success = df.groupby('main_category')['state'].apply(
    lambda x: (x == 'successful').sum() / len(x) * 100
).sort_values(ascending=False)

plt.figure(figsize=(14, 6))
sns.barplot(x=category_success.values, y=category_success.index, palette='coolwarm')
plt.title('Success Rate by Main Category (%)', fontsize=14, fontweight='bold')
plt.xlabel('Success Rate (%)')
plt.ylabel('Main Category')
plt.tight_layout()
plt.show()


In [ ]:
# 4. Top Countries by Project Count
plt.figure(figsize=(12, 6))
country_counts = df['country'].value_counts().head(15)
sns.barplot(x=country_counts.values, y=country_counts.index, palette='mako')
plt.title('Top 15 Countries by Project Count', fontsize=14, fontweight='bold')
plt.xlabel('Number of Projects')
plt.ylabel('Country')
plt.tight_layout()
plt.show()


In [ ]:
# 5. Distribution of Funding Goals (USD)
plt.figure(figsize=(14, 6))

# Filter out extreme outliers for better visualization
df_clean = df[df['usd_goal_real'] <= df['usd_goal_real'].quantile(0.95)]

plt.subplot(1, 2, 1)
plt.hist(df_clean['usd_goal_real'], bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of Funding Goals (USD)\n(95th percentile)', fontsize=12, fontweight='bold')
plt.xlabel('USD Goal')
plt.ylabel('Frequency')
plt.xscale('log')

plt.subplot(1, 2, 2)
plt.boxplot(df_clean['usd_goal_real'], vert=False)
plt.title('Box Plot of Funding Goals (USD)', fontsize=12, fontweight='bold')
plt.xlabel('USD Goal')
plt.xscale('log')

plt.tight_layout()
plt.show()

print(f"Median goal: ${df['usd_goal_real'].median():,.2f}")
print(f"Mean goal: ${df['usd_goal_real'].mean():,.2f}")


In [ ]:
# 6. Projects Launched Over Time
df_time = df.dropna(subset=['launched'])
df_time['year_month'] = df_time['launched'].dt.to_period('M')

monthly_counts = df_time['year_month'].value_counts().sort_index()

plt.figure(figsize=(16, 6))
plt.plot(monthly_counts.index.astype(str), monthly_counts.values, marker='o', linewidth=2, markersize=4)
plt.title('Number of Projects Launched Over Time', fontsize=14, fontweight='bold')
plt.xlabel('Year-Month')
plt.ylabel('Number of Projects')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# 7. Success Rate Over Time
df_time_success = df_time.groupby('year_month')['state'].apply(
    lambda x: (x == 'successful').sum() / len(x) * 100
).sort_index()

plt.figure(figsize=(16, 6))
plt.plot(df_time_success.index.astype(str), df_time_success.values, 
         marker='o', linewidth=2, markersize=4, color='green')
plt.title('Success Rate Over Time', fontsize=14, fontweight='bold')
plt.xlabel('Year-Month')
plt.ylabel('Success Rate (%)')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# 8. Campaign Duration Distribution
df_duration = df.dropna(subset=['campaign_duration_days'])
df_duration_clean = df_duration[
    (df_duration['campaign_duration_days'] >= 0) & 
    (df_duration['campaign_duration_days'] <= 90)
]

plt.figure(figsize=(12, 6))
plt.hist(df_duration_clean['campaign_duration_days'], bins=30, color='coral', edgecolor='black')
plt.title('Distribution of Campaign Duration (Days)', fontsize=14, fontweight='bold')
plt.xlabel('Campaign Duration (Days)')
plt.ylabel('Frequency')
plt.axvline(df_duration_clean['campaign_duration_days'].median(), 
            color='red', linestyle='--', linewidth=2, label=f'Median: {df_duration_clean["campaign_duration_days"].median():.0f} days')
plt.legend()
plt.tight_layout()
plt.show()

print(f"Median campaign duration: {df_duration_clean['campaign_duration_days'].median():.0f} days")
print(f"Mean campaign duration: {df_duration_clean['campaign_duration_days'].mean():.1f} days")


In [ ]:
# 9. Relationship between Goal and Pledged Amount (for successful projects)
df_successful = df[df['state'] == 'successful'].copy()
df_successful_clean = df_successful[
    (df_successful['usd_goal_real'] <= df_successful['usd_goal_real'].quantile(0.95)) &
    (df_successful['usd_pledged_real'] <= df_successful['usd_pledged_real'].quantile(0.95))
]

plt.figure(figsize=(12, 6))
plt.scatter(df_successful_clean['usd_goal_real'], df_successful_clean['usd_pledged_real'], 
           alpha=0.3, s=10)
plt.plot([0, df_successful_clean['usd_goal_real'].max()], 
         [0, df_successful_clean['usd_goal_real'].max()], 
         'r--', linewidth=2, label='Goal = Pledged')
plt.title('Goal vs Pledged Amount (Successful Projects)', fontsize=14, fontweight='bold')
plt.xlabel('USD Goal')
plt.ylabel('USD Pledged')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# 10. Backers Distribution
df_backers = df[df['backers'] > 0].copy()
df_backers_clean = df_backers[df_backers['backers'] <= df_backers['backers'].quantile(0.95)]

plt.figure(figsize=(12, 6))
plt.hist(df_backers_clean['backers'], bins=50, color='mediumpurple', edgecolor='black')
plt.title('Distribution of Number of Backers (95th percentile)', fontsize=14, fontweight='bold')
plt.xlabel('Number of Backers')
plt.ylabel('Frequency')
plt.xscale('log')
plt.tight_layout()
plt.show()

print(f"Median backers: {df['backers'].median():.0f}")
print(f"Mean backers: {df['backers'].mean():.1f}")
print(f"Max backers: {df['backers'].max():,}")


In [ ]:
# 11. Success Rate by Campaign Duration
df_duration_success = df_duration_clean.groupby(
    pd.cut(df_duration_clean['campaign_duration_days'], bins=10)
)['state'].apply(lambda x: (x == 'successful').sum() / len(x) * 100)

plt.figure(figsize=(12, 6))
sns.barplot(x=df_duration_success.index.astype(str), y=df_duration_success.values, palette='RdYlGn')
plt.title('Success Rate by Campaign Duration', fontsize=14, fontweight='bold')
plt.xlabel('Campaign Duration (Days)')
plt.ylabel('Success Rate (%)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


## 6. Key Insights and Summary


In [ ]:
# Summary statistics
print("=" * 60)
print("KEY INSIGHTS SUMMARY")
print("=" * 60)

print(f"\n1. Total Projects: {len(df):,}")
print(f"2. Successful Projects: {(df['state'] == 'successful').sum():,} ({(df['state'] == 'successful').sum() / len(df) * 100:.2f}%)")
print(f"3. Failed Projects: {(df['state'] == 'failed').sum():,} ({(df['state'] == 'failed').sum() / len(df) * 100:.2f}%)")

print(f"\n4. Top 3 Categories by Count:")
top_categories = df['main_category'].value_counts().head(3)
for cat, count in top_categories.items():
    success_rate = (df[df['main_category'] == cat]['state'] == 'successful').sum() / count * 100
    print(f"   - {cat}: {count:,} projects ({success_rate:.1f}% success rate)")

print(f"\n5. Top 3 Countries:")
top_countries = df['country'].value_counts().head(3)
for country, count in top_countries.items():
    success_rate = (df[df['country'] == country]['state'] == 'successful').sum() / count * 100
    print(f"   - {country}: {count:,} projects ({success_rate:.1f}% success rate)")

print(f"\n6. Funding Statistics:")
print(f"   - Median Goal: ${df['usd_goal_real'].median():,.2f}")
print(f"   - Median Pledged (successful): ${df[df['state'] == 'successful']['usd_pledged_real'].median():,.2f}")
print(f"   - Median Backers (successful): {df[df['state'] == 'successful']['backers'].median():.0f}")

if 'campaign_duration_days' in df.columns:
    df_dur = df.dropna(subset=['campaign_duration_days'])
    df_dur_clean = df_dur[(df_dur['campaign_duration_days'] >= 0) & (df_dur['campaign_duration_days'] <= 90)]
    print(f"\n7. Campaign Duration:")
    print(f"   - Median Duration: {df_dur_clean['campaign_duration_days'].median():.0f} days")
    print(f"   - Mean Duration: {df_dur_clean['campaign_duration_days'].mean():.1f} days")

print("\n" + "=" * 60)
